<a href="https://colab.research.google.com/github/ivs-math/watson/blob/master/Clasificador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Categorized Bin Packing 


---


This notebook discuses the solution to a particular bin packing problem that requires previous item classification. 

## Obective:
 **Determine the number of cold bags and baskets for each order.**

## Context
Today, orders are not being packed in a very efficient way, thus, the warehouse chief asked you to create an algorithm that can solve for any set of products how many baskets and cold bags are needed to correctly pack an order. 

## Conditions
Each order contains different products that have diverse characteristics.

Each of the products has:

**Dimensions** *(height, length, width)* in centimeters and *Weight* in grams

A **category** that determines if the product can be mixed with other products:

* *Food*

* *Toilet*

* *Pets*

A **storage type** that determines the type of package that will be used for the product

* *Dry*

* *Refrigerated*

* *Frozen*

There are some constraints associated with the type of package to be used when packing a product:

* All Dry products must be packed in a baskets

* All Refrigerated and Frozen products must be packed in a cold bag

* Products cannot be mixed if their categories are different, for example, a food product cannot be packed in the same basket that was used to pack a pet product, and also, a toilet product cannot be packed in the same basket that was used to pack a pet product, and so on.

The baskets have the following characteristics:

* Length: 50 centimeters

* Width: 40 centimeters

* Height: 60 centimeters

* Total weight that it can resit: 25 kilograms


The cold bags have the following characteristics:

* Length: 23 centimeters

* Width: 37 centimeters

* Height: 18 centimeters

* Total weight that it can resit: 5 kilograms

## Dataset

The given dataset is separated in different csv files:

* **orders**: it contains the orders that need the calculation of baskets and cold bags. Also, you can see when the order must be delivered and which warehouse received the order.

* **order_products**: contains what products and how many units of it were on an order from the file “orders”.

* **store_products**: contains information about the storage of the products and a marketing category.

* **products**: contains information about the products, its dimensions and weight, as well as their names and the can_mix category.




#**Solution**

---
\\
##**Bin Packing**

In order to solve the bin packing problem, we will use a *PypI* library named *binpacking*. This package contains greedy algorithms to solve two typical bin packing problems,  sorting items into a constant number of bins, and sorting items into a low number of bins of constant size.



> https://pypi.org/project/binpacking



This bin approximation will be used to distribute the volume from articles of a single category and storage type. Then, if any of the bin exceeds the maximum weight, the algorithm will take the heaviest item to a new bin.

This approximation prioritizes the volume over the weight, considering that it is more likely to exceed the maximum volume when optimizing the weight distribution.


In [8]:
!pip install binpacking

##**Classification**

Before any bin is created ,we need to take all the items in an order and classify them in one of the following groups:

* Dry-Pets (DP)
* Dry-Toilet (DT)
* Dry-Food (DF)
* Refrigerated-Pets (RP)
* Refrigerated-Toilet (RT)
* Refrigerated-Food (RF)

The bin packing algorithm will be applied only after this classification is complete.

From now we will solve the problem for the order **1781560**. In the final part we will generalize the solution for any given order.

> We define a pandas dataframe for each csv. This will allow us to access the data efficiently.

In [128]:
import pandas as pd
df_products=pd.read_csv('products.csv')
df_order_prod=pd.read_csv('order_products.csv')
df_orders=pd.read_csv('orders.csv')
df_store_prod=pd.read_csv('store_products.csv')

>  Let's take all the products in order_products with the same order_id.

In [136]:
df_list=df_order_prod[df_order_prod['order_id']==1781562]
df_list=df_list.reset_index(drop=True)

> Then we will classify the items in one of the six groups

In [137]:
#Classify the Storage

Storage=[]
for x in df_list['store_product_id']:
  z=df_store_prod.loc[df_store_prod['store_product_id'] == x]
  l=z['storage'].values.tolist()
  Storage.append(l[0])

df_list.insert(1,"Storage",Storage)


In [138]:
#Classify the Category
Prod_id=[]

for x in df_list['store_product_id']:
  z=df_store_prod.loc[df_store_prod['store_product_id'] == x]
  l=z['product_id'].values.tolist()
  Prod_id.append(l[0])

df_list.insert(1,"product_id",Prod_id)

Mix=[]
for x in df_list['product_id']:
  z=df_products.loc[df_products['product_id'] == x]
  l=z['can_mix'].values.tolist()
  Mix.append(l[0])

df_list.insert(1,"can_mix",Mix)


In [139]:
st=df_list['Storage'].values.tolist()
mx=df_list['can_mix'].values.tolist()
group=[]
for (x, y) in zip(st, mx):
  if (x,y)==('Seco','Food'):
    group.append('DF')
  if (x,y)==('Seco','Toilet'):
    group.append('DT')
  if (x,y)==('Seco','Pets'):
    group.append('DP')
  if ((x,y)==('Congelado','Food')) or ((x,y)==('Refrigerado','Food')):
    group.append('RF')
  if ((x,y)==('Congelado','Toilet')) or ((x,y)==('Refrigerado','Toilet')):
    group.append('RT')
  if ((x,y)==('Congelado','Pets')) or ((x,y)==('Refrigerado','Pets')):
    group.append('RP')


df_list.insert(1,"group",group)

df_list


    order_id group can_mix  product_id      Storage  store_product_id  \
0    1781562    DF    Food    181808.0         Seco             16118   
1    1781562    DF    Food      1320.0         Seco               647   
2    1781562    DF    Food      1877.0         Seco               742   
3    1781562    RF    Food    173582.0  Refrigerado              1170   
4    1781562    DF    Food    174323.0         Seco              4542   
5    1781562    RF    Food    172897.0  Refrigerado               879   
6    1781562    RF    Food    172898.0  Refrigerado               880   
7    1781562    RF    Food    172955.0  Refrigerado               884   
8    1781562    DF    Food    178184.0         Seco             10414   
9    1781562    DT  Toilet    178230.0         Seco             10490   
10   1781562    DF    Food    180965.0         Seco             14931   
11   1781562    DF    Food    173661.0         Seco              1249   
12   1781562    DT  Toilet    175734.0         Seco

>Now that each item is classified, we will calculate its weight and volume.

In [140]:
volume=[]
weight=[]
for x in df_list['product_id']:
  z=df_products.loc[df_products['product_id'] == x]
  len=z['length'].values.tolist()
  wid=z['width'].values.tolist()
  hei=z['height'].values.tolist()
  wei=z['weight'].values.tolist()
  vol=len[0]*wid[0]*hei[0]
  volume.append(vol)
  weight.append(wei[0])

df_list.insert(1,"volume",volume)
df_list.insert(1,"weight",weight)


df_list


,order_id,weight,volume,group,can_mix,product_id,Storage,store_product_id,quantity
0,1781562,2500.000,36046.08,DF,Food,181808.0,Seco,16118,1
1,1781562,508.000,2730.00,DF,Food,1320.0,Seco,647,2
2,1781562,1628.000,2673.00,DF,Food,1877.0,Seco,742,1
3,1781562,492.000,700.00,RF,Food,173582.0,Refrigerado,1170,1
4,1781562,418.000,1664.00,DF,Food,174323.0,Seco,4542,1
5,1781562,1022.000,1560.00,RF,Food,172897.0,Refrigerado,879,1
6,1781562,1022.000,1560.00,RF,Food,172898.0,Refrigerado,880,1
7,1781562,1022.000,1560.00,RF,Food,172955.0,Refrigerado,884,2
8,1781562,6820.000,8280.00,DF,Food,178184.0,Seco,10414,1
9,1781562,0.373,550.00,DT,Toilet,178230.0,Seco,10490,1


>Now we have everything we need from the files we can drop the unused columns.

In [145]:
df_list=df_list.drop(['order_id', 'can_mix','product_id','Storage','store_product_id'], axis=1)

df_list

,weight,volume,group,quantity
0,2500.000,36046.08,DF,1
1,508.000,2730.00,DF,2
2,1628.000,2673.00,DF,1
3,492.000,700.00,RF,1
4,418.000,1664.00,DF,1
5,1022.000,1560.00,RF,1
6,1022.000,1560.00,RF,1
7,1022.000,1560.00,RF,2
8,6820.000,8280.00,DF,1
9,0.373,550.00,DT,1


In [151]:
dic=df_list.to_dict()

dic['group'][3]

'RF'